In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from feature_extraction import Blob, Words
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neural_network import MLPClassifier

In [63]:
data = pd.read_csv('processed_posts.csv.bz2')
#data = pd.read_csv('processed_datascience.csv.bz2', encoding='ISO-8859-1', compression='bz2')
data.drop_duplicates('title', inplace=True)

data['gt5'] = data['ups'] > 5
data['gt20'] = data['ups'] > 20
data['gt10'] = data['ups'] > 10
data['gt50'] = data['ups'] > 50
data['gt100'] = data['ups'] > 100
# data['wordcount'] = data['title'].str.split().apply(len)
# data['charcount'] = data.title.str.len()
# data['vowels'] = data.title.str.findall(r'[aeiou]').apply(len)
# data['consonants'] = data.title.str.findall(r'[^aeiou]').apply(len)

#data = data[data['subreddit'] == 'r/books']
train_X, test_X, train_y, test_y = train_test_split(data.title, 
                                                    data.gt10, 
                                                    test_size=0.20,
                                                    random_state=42)

In [64]:
print(data[data['gt10']==False].shape)
print(data[data['gt10']==True].shape)


(115507, 11)
(138308, 11)


In [65]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('words', Words()),
                      
            ('title', Pipeline([
                ('tfidf', TfidfVectorizer(ngram_range=(1,3), sublinear_tf=True, stop_words='english')),
                ('svd', TruncatedSVD(n_components=120)),
                ('normalize', MinMaxScaler(copy=False)),
                ('selector', SelectPercentile(f_classif, percentile=10))
            ])),
            
            ('blob', Pipeline([
                ('all', Blob()),
                ('minmax', MinMaxScaler()),
            ])),
            
            ])),
    ('clf', MLPClassifier()),
        ])

In [66]:
pipeline.fit(train_X, train_y)

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('words', Words()), ('title', Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))])

In [59]:
y = pipeline.predict(test_X)
accuracy_score(y_pred=y, y_true=test_y)

0.78099938309685379

In [ ]:
cross_val_score(pipeline, train_X, train_y, cv=5)

In [62]:
index = 2
print (test_X.iloc[index])
print (test_y.iloc[index])
print (y[index])
# pipeline.predict(pd.Series(['My Job Search as a PhD Student']))[0]

b'We shall miss Hans Rosling, but let\xe2\x80\x99s learn from his genius'
True
False


In [ ]:
data.to_csv('gt_politics.csv', index=False)

In [ ]:
pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[
                      
            ('wordcount', WordCount()),
            
            ('charcount', CharCount()),
            
            ('title', TfidfVectorizer(ngram_range=(1,4))),
            
            ('vowel', Vowels()),
            
            ('consonants', Consonants()),
            
            ('polarity', Polarity()),
            
            ('subjectivity', Subjectivity()),
            
            ('noun_phrases', Nouns()),
            
            ])),
    ('clf', ExtraTreesClassifier()),
        ])